In [1]:
!pip install matplotlib numpy pandas


You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
import yaml
import datetime
import time

from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
results_dir = Path("results")
nodes_file = results_dir / "nodes.yaml"

with nodes_file.open("r") as f:
    node_descriptors = yaml.load(f, Loader=yaml.FullLoader)
    
print(f"There are {len(node_descriptors)} nodes")

There are 15 nodes


In [4]:
node_descriptors[0]

{'cloud_instance_id': 'i-046310ea085e2ae4d',
 'cloud_lifecycle': 'normal',
 'configuration': {'instance': {'boot_disk_device': None,
   'boot_disk_iops': None,
   'boot_disk_size': 16,
   'boot_disk_snapshot': None,
   'boot_disk_type': None,
   'flavor': 't2.medium',
   'image_id': 'ami-0650ed74180c860f3',
   'instance_config_id': 'lammps-type-t2.medium',
   'login': 'login-ubuntu',
   'network_ids': [],
   'placement_group': None,
   'price': None,
   'provider': 'aws-config-us-east-1',
   'security_group': 'open-security-group',
   'timeout': None},
  'login': {'keypair_name': 'otavio_aws_2022_key',
   'keypair_private_file': 'key.pem',
   'keypair_public_file': 'key.pub',
   'login_config_id': 'login-ubuntu',
   'ssh_port': 22,
   'sudo': True,
   'sudo_user': 'root',
   'user': 'ubuntu'},
  'provider': {'access_keyfile': 'ec2_access_key.pub',
   'provider': 'aws',
   'provider_config_id': 'aws-config-us-east-1',
   'region': 'us-east-1',
   'secret_access_keyfile': 'ec2_access_key

In [5]:
def get_lammps_time(file: Path):
    with file.open("r") as f:
        last_line = f.readlines()[-1]
    
    wall_time = last_line.split("Total wall time: ")[1].strip()
    wall_time = time.strptime(wall_time, "%H:%M:%S")
    wall_time = datetime.timedelta(
        hours=wall_time.tm_hour,
        minutes=wall_time.tm_min,
        seconds=wall_time.tm_sec).total_seconds()
    return int(wall_time)

results = {
    result_file.parents[3].stem: get_lammps_time(result_file)
    for result_file in results_dir.rglob("*.stdout") 
}

results

{'fcd71ec358ca49bcb6e43257a90b5b68': 119,
 '5b9555a28ceb47e0b64aa6eafe108c9a': 80,
 '7b30252402984d268d838fc9a8468fb7': 128,
 '419951bb461247c68b34115e596fb2da': 69,
 '98eaa8f47b444cf3afc295102886c2ac': 69,
 'b73ea280daca4f91a50b13844ba26b3c': 31,
 '1c674064c6324a90a13bf6b092afd80a': 78,
 '9b52a6a911cd401bb4c0e7985a12089a': 28,
 'bd768c6a66f445e7a11351a667e42b8a': 61,
 'dc8897c9bd31486dab80009367ef3385': 62,
 '4845d9798a284a6c9186c949c3cbb3d4': 203,
 '17505645616247a79c8f70ffd08778c1': 83,
 '055a1b8ed33e4070b12f8544aea86cd2': 150,
 'af385a34e484447bbad51afad3beb5c8': 168,
 '59794c67db354b7f87340624c04e4dd0': 174}

In [6]:
wall_time_per_flavor = {}
for k, v in results.items():
    flavor = [node["configuration"]["instance"]["flavor"] for node in node_descriptors if node["node_id"] == k][0]
    wall_time_per_flavor[flavor] = v
    
wall_time_per_flavor

{'r5.xlarge': 119,
 'm4.2xlarge': 80,
 'r5a.xlarge': 128,
 'm5a.2xlarge': 69,
 'r5a.2xlarge': 69,
 'r5.4xlarge': 31,
 'r4.2xlarge': 78,
 'c5.4xlarge': 28,
 'm5.2xlarge': 61,
 'r5.2xlarge': 62,
 'c5.large': 203,
 'x1e.2xlarge': 83,
 'r4.xlarge': 150,
 't2.large': 168,
 't2.medium': 174}

In [7]:
instance_price_per_second = {
    't2.medium':    0.0464/3600,
    'r4.2xlarge':   0.5320/3600,
    'r5.xlarge':    0.2520/3600,
    'm4.2xlarge':   0.4000/3600,
    'm5.2xlarge':   0.3840/3600,
    'r5a.xlarge':   0.2260/3600,
    'c5.4xlarge':   0.6800/3600,
    'r5a.2xlarge':  0.4520/3600,
    'r5.4xlarge':   1.0080/3600,
    'r5.2xlarge':   0.5040/3600,
    't2.large':     0.0928/3600,
    'm5a.2xlarge':  0.3440/3600,
    'c5.large':     0.0850/3600,
    'r4.xlarge':    0.2660/3600,
    'x1e.2xlarge':  1.6680/3600
}

instance_price_per_second

{'t2.medium': 1.2888888888888887e-05,
 'r4.2xlarge': 0.0001477777777777778,
 'r5.xlarge': 7e-05,
 'm4.2xlarge': 0.00011111111111111112,
 'm5.2xlarge': 0.00010666666666666667,
 'r5a.xlarge': 6.277777777777778e-05,
 'c5.4xlarge': 0.0001888888888888889,
 'r5a.2xlarge': 0.00012555555555555557,
 'r5.4xlarge': 0.00028,
 'r5.2xlarge': 0.00014,
 't2.large': 2.5777777777777775e-05,
 'm5a.2xlarge': 9.555555555555554e-05,
 'c5.large': 2.3611111111111114e-05,
 'r4.xlarge': 7.38888888888889e-05,
 'x1e.2xlarge': 0.00046333333333333334}

In [8]:
execution_cost = {
    instance: wall_time_per_flavor[instance]*instance_price_per_second[instance]
    for instance in instance_price_per_second.keys()
}

execution_cost

{'t2.medium': 0.0022426666666666662,
 'r4.2xlarge': 0.011526666666666668,
 'r5.xlarge': 0.008329999999999999,
 'm4.2xlarge': 0.008888888888888889,
 'm5.2xlarge': 0.006506666666666667,
 'r5a.xlarge': 0.008035555555555556,
 'c5.4xlarge': 0.00528888888888889,
 'r5a.2xlarge': 0.008663333333333334,
 'r5.4xlarge': 0.008679999999999998,
 'r5.2xlarge': 0.008679999999999998,
 't2.large': 0.004330666666666666,
 'm5a.2xlarge': 0.006593333333333332,
 'c5.large': 0.0047930555555555565,
 'r4.xlarge': 0.011083333333333334,
 'x1e.2xlarge': 0.03845666666666667}

In [9]:
sorted(execution_cost.items(), key=lambda x: x[1], reverse=False) 

[('t2.medium', 0.0022426666666666662),
 ('t2.large', 0.004330666666666666),
 ('c5.large', 0.0047930555555555565),
 ('c5.4xlarge', 0.00528888888888889),
 ('m5.2xlarge', 0.006506666666666667),
 ('m5a.2xlarge', 0.006593333333333332),
 ('r5a.xlarge', 0.008035555555555556),
 ('r5.xlarge', 0.008329999999999999),
 ('r5a.2xlarge', 0.008663333333333334),
 ('r5.4xlarge', 0.008679999999999998),
 ('r5.2xlarge', 0.008679999999999998),
 ('m4.2xlarge', 0.008888888888888889),
 ('r4.xlarge', 0.011083333333333334),
 ('r4.2xlarge', 0.011526666666666668),
 ('x1e.2xlarge', 0.03845666666666667)]

In [10]:
def select_pareto_frontier_points(Xs, Ys, maxX=True, maxY=True):
    '''Pareto frontier selection process'''
    sorted_list = sorted([[Xs[i], Ys[i]] for i in range(len(Xs))], reverse=maxY)
    pareto_front = [sorted_list[0]]
    for pair in sorted_list[1:]:
        if maxY:
            if pair[1] >= pareto_front[-1][1]:
                pareto_front.append(pair)
        else:
            if pair[1] <= pareto_front[-1][1]:
                pareto_front.append(pair)
    return pareto_front

findfont: Font family ['serif'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'serif' not found because none of the following families were found: Times
findfont: Font family ['Times'] not found. Falling back to DejaVu Sans.


In [11]:
# Try to use a serif font to make labels more similar to latex text.
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rc('font',**{'family':'serif','serif':['Times']})


instances = sorted(execution_cost.keys())
costs = np.array([(execution_cost[i],wall_time_per_flavor[i]) for i in instances])
paretto_frontier_instance = ['t2.medium','t2.large','c5.4xlarge']
paretto_points = [i in paretto_frontier_instance for i in instances] 
paretto_colors = [ "red" if i else "blue" for i in paretto_points ]

xs = [execution_cost[i] for i in instances]
ys = [wall_time_per_flavor[i] for i in instances]

# Collect pareto frontier points to draw dashed line
pareto_front = select_pareto_frontier_points(xs, ys, maxX=False, maxY=False)
pf_X = [pair[0] for pair in pareto_front]
pf_Y = [pair[1] for pair in pareto_front]
pf_X.insert(0,pf_X[0])
pf_Y.insert(0,max(ys))
pf_X.append(max(xs))
pf_Y.append(pf_Y[-1])

markers = ["o", "v", "^", "<", ">", "1", "2", "8", "s", "h", "P", "p", "*", "D", "d"]*2
fig, ax = plt.subplots(figsize=(8, 6))

ax.plot(pf_X, pf_Y,color='gray', linestyle='dashed')

for label, x, y, marker, color in zip(instances, xs, ys, markers, paretto_colors):
    ax.scatter(x, y, label=label, marker=marker, c=color, alpha=0.75)
    
ax.set_xlabel("Cost (USD)",fontname="Times")
ax.set_ylabel("Execution Time (s)")

ax.legend()
plt.savefig("results/instances-cost-lammps-rhodo.pdf", bbox_inches="tight") 
plt.show()

In [12]:
i1 = "x1e.2xlarge"
i2 = "c5.4xlarge"
print("{} vs {}".format(i1,i2))
print("Execution cost: {}/{} = {}".format(i1,i2,execution_cost[i1]/execution_cost[i2]))
print("Wallclock time: {}/{} = {}".format(i1,i2,wall_time_per_flavor[i1]/wall_time_per_flavor[i2]))


x1e.2xlarge vs c5.4xlarge
Execution cost: x1e.2xlarge/c5.4xlarge = 7.271218487394957
Wallclock time: x1e.2xlarge/c5.4xlarge = 2.9642857142857144
